In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install langchain_community

In [3]:
! pip install numexpr


In [4]:
! pip install openai

In [5]:
#!pip install langchain openai
from langchain import OpenAI, LLMMathChain
llm = OpenAI(temperature=0,\
             openai_api_key="XXXXXX")

/home/ntadmin/ntadminenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
from langchain.agents import Tool, initialize_agent

In [7]:
llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [8]:
def check_grammar(text: str) -> str:
    # Use a grammar check API or library here
    return f"Grammar checked: {text}"
grammar_tool = Tool(
    name="Check Grammar",
    func=check_grammar,
    description="Check the grammar of English text."
)


In [9]:
def summarize_text(text: str) -> str:
    # Use a summarization model here
    return f"Summary: {text[:50]}..."
summarize_tool = Tool(
    name="Summarize Text",
    func=summarize_text,
    description="Summarize long text content."
)


In [10]:
math_tool = Tool(
    name="Calci",
    func=llm_math.run,
    description="Useful for when you need to answer questions related to Math."
)


grammar_tool = Tool(
    name="Check Grammar",
    func=check_grammar,
    description="Check the grammar of English text."
)

summarize_tool = Tool(
    name="Summarize Text",
    func=summarize_text,
    description="Summarize long text content."
)

tools = [math_tool, grammar_tool, summarize_tool]

In [11]:
agent = initialize_agent(
    agent="zero-shot-react-description",
    agent_type="chat-conversational",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3

)

## print(agent)


/home/ntadmin/ntadminenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [12]:
# Math tool
response = agent("What is (4*512)+152")
print(response)


# Grammar check tool
response = agent("Check the grammar of this sentence: 'I goes to school.'")
print(response)

# Summarize tool
response = agent("""
Summarize this text: 
Artificial Intelligence (AI) is a field of study that focuses on 
creating systems capable of performing tasks that require human intelligence. 
These tasks include learning, reasoning, problem-solving, perception, 
and language understanding.
""")
print(response)


/home/ntadmin/ntadminenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Calci to solve this math problem.
Action: Calci
Action Input: (4*512)+152

> Entering new LLMMathChain chain...
(4*512)+152```text
(4*512)+152
```
...numexpr.evaluate("(4*512)+152")...

Answer: 2200
> Finished chain.

Observation: Answer: 2200
Thought: I now know the final answer.
Final Answer: 2200

> Finished chain.
{'input': 'What is (4*512)+152', 'output': '2200'}


> Entering new AgentExecutor chain...
 I should use the Check Grammar tool to check the grammar of the sentence.
Action: Check Grammar
Action Input: 'I goes to school.'
Observation: Grammar checked: 'I goes to school.'
Thought: I now know the final answer
Final Answer: The sentence 'I goes to school.' is grammatically incorrect.

> Finished chain.
{'input': "Check the grammar of this sentence: 'I goes to school.'", 'output': "The sentence 'I goes to school.' is grammatically incorrect."}


> Entering new AgentExecutor chain...
 I should use the Summarize Text tool to